In [1]:


"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding more
average features and weekly average features on it.
"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import sys

import math
import sklearn.metrics as skl_metrics

from datetime import timedelta
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

df_train = pd.read_csv(
    '../input/train.csv', usecols=[0, 1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    parse_dates=["date"],
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
)


df_train_maxdate = df_train[['item_nbr','store_nbr','date']].groupby(['item_nbr','store_nbr'])\
    ['date'].max().to_frame('max_date')

/home/zyp/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/zyp/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_train_maxdate.shape

(174685, 1)

In [3]:
df_train_maxdate.reset_index(inplace=True)

del df_train



In [4]:
## Discontinue store/items
df_dis = df_train_maxdate.loc[df_train_maxdate['max_date'] <= '2017-05-16',]


In [8]:
df_dis.max_date.max()

Timestamp('2017-05-16 00:00:00')

In [10]:
t_prev = pd.read_csv('../submit/T007_MoreWKs_TrainToAug15.csv')
df_prev = pd.merge(t_prev, df_test,  on=['id'], how = 'inner')


In [12]:
df_prev.head(2)

,id,unit_sales,date,store_nbr,item_nbr,onpromotion
0,125497040,0.2107,2017-08-16,1,96995,False
1,125497041,0.4035,2017-08-16,1,99197,False


In [18]:
df_sub = pd.merge(df_prev, df_dis,  on=['store_nbr', 'item_nbr'], how = 'inner')


In [19]:
df_sub.unit_sales = 0 

In [20]:

t_new = pd.merge(df_prev, df_sub,  on=['id'], how = 'left')
t_new['unit_sales'] = t_new.unit_sales_y.combine_first(t_new.unit_sales_x)


In [22]:

submission = t_new[['id','unit_sales']]
#del t_new

print("Merged Partial, SUM =",  submission.unit_sales.sum())
print("Merged Partial, MEAN =",  submission.unit_sales.mean())

submission.to_csv('../submit/T013_zero_3m_discontiue.csv', float_format='%.4f', index=None)


Merged Partial, SUM = 9939036.979100117
Merged Partial, MEAN = 2.9488631176894686


In [24]:
df_sub.head()

,id,unit_sales,date,store_nbr,item_nbr,onpromotion,max_date
0,125497060,0,2017-08-16,1,108831,False,2017-02-11
1,125707714,0,2017-08-17,1,108831,False,2017-02-11
2,125918368,0,2017-08-18,1,108831,False,2017-02-11
3,126129022,0,2017-08-19,1,108831,False,2017-02-11
4,126339676,0,2017-08-20,1,108831,False,2017-02-11
